# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from graph_representation_generator import GraphRepresentationGenerator
from dataset_manager import (
    MovieLensManager,
    INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
)

In [2]:
EPOCHS = 20
BATCH_SIZE_KGE = 128000
BATCH_SIZE_LLM = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

Using existing file ml-32m.zip
Extracting ./data\ml-32m.zip


splitting LLM dataset


First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-32m.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [4]:
kg_manager.data

HeteroData(
  source={ node_id=[200948] },
  target={
    node_id=[87585],
    x=[87585, 20],
  },
  (source, edge, target)={ edge_index=[2, 32000204] },
  (target, rev_edge, source)={ edge_index=[2, 32000204] }
)

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
graph_representation_generator_graph_prompter_hf = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    hidden_channels=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    kge_dimension=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    force_recompute=False,
)

Device: 'cpu'


We then train and validate the model on the link prediction task.

If the model is already trained, we can skip this part.
Training the models can take up to 5 minutes.

In [10]:
print("GraphPrompterHF Training")
graph_representation_generator_graph_prompter_hf.train_model(
    kg_manager.gnn_train_data, EPOCHS, BATCH_SIZE_KGE
)
graph_representation_generator_graph_prompter_hf.validate_model(
    kg_manager.gnn_test_data, batch_size=BATCH_SIZE_KGE
)


GraphPrompterHF Training


100%|██████████| 201/201 [37:43<00:00, 11.26s/it]


Epoch: 001, Loss: 0.2283


100%|██████████| 201/201 [37:57<00:00, 11.33s/it]


Epoch: 002, Loss: 0.1375


...

Epoch: 019, Loss: 0.0742


100%|██████████| 201/201 [38:12<00:00, 11.41s/it]


Epoch: 020, Loss: 0.0740


100%|██████████| 51/51 [04:54<00:00,  5.77s/it]



Validation AUC: 0.9944


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.